In [ ]:
# We install the package through devtools::install_github
# This requires the "devtools" package to be installed
install.packages("devtools",  INSTALL_opts = c('--no-lock'))
library(devtools)
devtools::install_github("TobiasRoikjer/PtDAlgorithms")

In [1]:
# Import the functions in the library
library(ptdalgorithms)

# The library supports setting seeds for some of its
# sampling capability, as any other R package
set.seed(1234)

In [6]:
# While the R api can create graphs, this is *slow* as the C++ binding layer
# through Rcpp is slow when invoking many functions, and since R is a slow,
# interpreted language
# Clone or download the code, and include these files in the repository!
# Make SURE that the version of the downloaded code is the same as the
# installed R library!! Otherwise it may crash randomly
# The file has comments and is easy to understand, so you should be able
# to defined you own cool construction functions

Rcpp::sourceCpp("./isolation_migration_model.cpp")

In [ ]:
# Comparing results to verify implementation/numerical accuracy
n1 <- 4
n2 <- 4

g <- construct_im_graph(n1,n2,0.1,0.1)

expected_visits <- rep(0, vertices_length(g))
ctx <- distribution_context(g,1000)
while (distribution_context_state(ctx)$cdf < 0.9999) {
  distribution_context_step(ctx)
}

expected_visits <- distribution_context_accumulated_visiting_time(ctx)

distribution_expectation <- matrix(nrow=n1+1,ncol=n2+1)
algorithm_expectation <- matrix(nrow=n1+1,ncol=n2+1)
matrix_expectation <- matrix(nrow=n1+1,ncol=n2+1)
simulation_expectation <- matrix(nrow=n1+1,ncol=n2+1)
PH <- graph_as_matrix(g)
U <- solve(-PH$SIM)
set.seed(1234)

for (i in 0:n1) {
  for (j in 0:n2) {
    distribution_expectation[i+1,j+1] <- sum(expected_visits * rewards_at(g, i,j,n1,n2))
    algorithm_expectation[i+1, j+1]<- expectation(g, rewards_at(g, i,j,n1,n2))
    simulation_expectation[i+1, j+1]<- mean(rph(10000, g, rewards_at(g, i,j,n1,n2)))
    matrix_expectation[i+1,j+1] <-PH$IPV %*% U%*%diag(PH$states[,(matrix_index(i,j,0,n1,n2)+1)]+PH$states[,(matrix_index(i,j,1,n1,n2)+1)])%*%rep(1,length(PH$IPV))
  }
}

sum(abs(matrix_expectation - algorithm_expectation))
sum(abs(matrix_expectation - simulation_expectation))
sum(abs(matrix_expectation - distribution_expectation))


In [ ]:

###
### Timings of 8x8 expectations
###

n1 <- 8
n2 <- 8

time <- proc.time()[3]
g <- construct_im_graph(n1,n2,0.1,0.1)

cat(paste("Construction took ", proc.time()[3] - time, "seconds\n"))

expected_visits <- rep(0, vertices_length(g))
time <- proc.time()[3]
ctx <- distribution_context(g,1000)
while (distribution_context_state(ctx)$time <= 1.5) {
    distribution_context_step(ctx)
}

cat(paste("Finding stopping time took ", proc.time()[3] - time, "seconds\n"))


algorithm_expectation <- matrix(nrow=n1+1,ncol=n2+1)

time <- proc.time()[3]
k <- expectation(g)

cat(paste("Moment graph took ", proc.time()[3] - time, "seconds\n"))

time <- proc.time()[3]
for (i in 0:n1) {
  for (j in 0:n2) {
    algorithm_expectation[i+1, j+1]<- expectation(g, rewards_at(g, i,j,n1,n2))
    }
}

cat(paste("Expectations took ", proc.time()[3] - time, "seconds\n"))